In [1]:
import pandas as pd
import numpy as np

import patsy as pt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import plotly.express as px

from interpret import show
from interpret.blackbox import ShapKernel, PartialDependence



### Data Prep

In [2]:
data = pd.read_csv('./DATA/pakistanClean2.csv')

In [3]:
#valid_train_Values = ['0','1']
#work_data = data[data.TTP.isin(valid_train_Values)] 
work_data = data[(data['TTP']==0) | (data['TTP'] ==1)].copy()
test_data = data[pd.isna(data['TTP'])].copy()
work_data = work_data.fillna(-99)

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [4]:
# Create regression arrays
Y, X = pt.dmatrices("TTP ~ C(iyear) + C(provstate) + multiple + success + suicide + attacktype1 + C(targtype1) + C(targsubtype1) + weaptype1 + weapsubtype1 + nkill + nkillus + nkillter + nwound + nwoundus + nwoundte + C(Month)", data = work_data, return_type='dataframe')

In [5]:
names = X.columns
names = [i.replace('[', '').replace(']', '').replace(' ', '').replace(',', '') for i in names]

In [6]:
# Randomly create train and test data
x, xt, y, yt = train_test_split(X, Y, test_size = 0.25,random_state=35)

### Model

In [7]:
# Generate the random forest model
writingForest = RandomForestClassifier(n_estimators=110, n_jobs = -1, random_state=35)
# Fit the model to the training data
fclf = writingForest.fit(x, y)

<ipython-input-7-4e61eff9e664>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fclf = writingForest.fit(x, y)


### Generate predictions and evaluate

In [8]:
# Make predictions
fpred = fclf.predict(xt)
# Print the accuracy score of the fitted model
print("The random forest has an accuracy of : %s\n" % str(accuracy_score(fpred, yt)))

The random forest has an accuracy of : 0.8175937904269082



### Explain model

#### Feature importances

In [9]:
imp_df = pd.DataFrame({'Feature':names,'Importance':fclf.feature_importances_})
imp_df.sort_values(by='Importance', ascending=False, inplace=True)

In [10]:
px.bar(x='Importance', y='Feature', data_frame=imp_df.head(20), orientation='h')

#### Shapley Values Intro

Shapley values can help us understand what and how features contributed to our final prediction. The goal is to understand how a model got to the prediction it did. In the above plot we see that the number of terrorists killed, along with a couple location features, pushed our TTP probabibility over 50%. While there were other features that would lead us to believe it was not a TTP attack, they were not enough to bring us below 50%. 

In [11]:
shap = ShapKernel(predict_fn=writingForest.predict_proba, data=x[:100])

In [12]:
shap_local = shap.explain_local(pd.DataFrame(xt)[:25], pd.DataFrame(yt)[:25])

show(shap_local)

  0%|          | 0/25 [00:00<?, ?it/s]num_full_subsets = 1
remaining_weight_vector = [0.12794589 0.08633747 0.06555252 0.05309754 0.04480805 0.0388993
 0.03447893 0.0310512  0.02831869 0.02609216 0.02424545 0.02269126
 0.02136722 0.02022764 0.01923824 0.01837285 0.01761113 0.01693702
 0.01633771 0.01580284 0.01532397 0.01489412 0.01450753 0.01415935
 0.01384551 0.01356259 0.01330766 0.01307821 0.01287213 0.01268759
 0.01252301 0.01237705 0.01224857 0.01213658 0.01204026 0.01195891
 0.01189195 0.01183892 0.01179945 0.01177329 0.01176025]
num_paired_subset_sizes = 42
weight_left = 0.7981830466727807
np.sum(w_aug) = 85.0
np.sum(self.kernelWeights) = 1.0
phi = [-0.00291031 -0.00471585 -0.00286226  0.          0.          0.
  0.00666694  0.          0.0047356  -0.0101411   0.          0.11185225
 -0.02815481  0.         -0.03486732  0.          0.          0.
  0.00914059 -0.00738744  0.          0.          0.          0.00323748
  0.          0.00371935  0.          0.0086892   0.       

<!-- http://127.0.0.1:7001/1500870105504/ -->

#### Partial Dependence Plot Intro
Partial dependence plots reveal the dependence between our target variable and any given feature.  

In [14]:
pdp = PartialDependence(predict_fn=writingForest.predict_proba, data=x, num_points=100)

In [15]:
pdp_global = pdp.explain_global()

show(pdp_global)

Generating mini dash
Generated mini dash


<!-- http://127.0.0.1:7001/1498994969472/ -->

### Generate predictions on true test dataset

In [16]:
test_data.fillna(-99, inplace=True)

In [17]:
# Create regression arrays
_, Xtest = pt.dmatrices("TTP ~ C(iyear) + C(provstate) + multiple + success + suicide + attacktype1 + C(targtype1) + C(targsubtype1) + weaptype1 + weapsubtype1 + nkill + nkillus + nkillter + nwound + nwoundus + nwoundte + C(Month)", data = test_data, return_type='dataframe')

In [18]:
X['C(provstate)[T.Balochistan]'].mean()

0.34833279378439624

In [19]:
Xtest['C(provstate)[T.Balochistan]'].mean()

0.28115435067774375

In [20]:
remove_cols = set(Xtest.columns) - set(X.columns)
add_cols = set(X.columns) - set(Xtest.columns)

No overall plot to display: -1|PartialDependence_0


In [21]:
for col in remove_cols:
    del Xtest[col]
for col in add_cols:
    Xtest[col] = 0

In [22]:
# Make predictions
fpred = fclf.predict(Xtest)

In [23]:
pred_df = pd.DataFrame({'eventid':test_data['eventid'], 'TTP':fpred})
pred_df.head()

,eventid,TTP
0,200712030005,0.0
1,200712040005,0.0
3,200712080003,0.0
4,200712090002,0.0
5,200712090004,0.0


In [24]:
pred_df.to_csv('test_preds.csv')

In [25]:
pred_df['TTP'].mean()

0.5124617402710975

In [26]:
shap_local = shap.explain_local(pd.DataFrame(Xtest)[:25])

show(shap_local)

  0%|          | 0/25 [00:00<?, ?it/s]num_full_subsets = 1
remaining_weight_vector = [0.12679607 0.08554916 0.06494433 0.05259689 0.04437863 0.03852033
 0.0341374  0.03073844 0.02802861 0.02582029 0.02398845 0.02244651
 0.02113268 0.02000163 0.01901941 0.01816005 0.01740338 0.0167335
 0.01613768 0.01560567 0.01512908 0.01470099 0.01431569 0.01396835
 0.01365496 0.01337209 0.01311684 0.01288672 0.01267961 0.01249369
 0.0123274  0.01217938 0.0120485  0.01193375 0.0118343  0.01174944
 0.01167859 0.01162124 0.01157703 0.01154566 0.01152692 0.01152068]
num_paired_subset_sizes = 42
weight_left = 0.7986833459821002
np.sum(w_aug) = 86.0
np.sum(self.kernelWeights) = 1.0
phi = [-0.00389437 -0.00525297 -0.00818692  0.          0.          0.
  0.          0.          0.          0.          0.         -0.23063017
 -0.04021534  0.         -0.03396033 -0.00304347  0.          0.
 -0.00276195 -0.00275979  0.          0.          0.          0.
  0.          0.          0.          0.          0.    

<!-- http://127.0.0.1:7001/1498995046864/ -->

No overall plot to display: -1|ShapKernel_1
